https://velog.io/@kite_day/colab-%EC%97%90%EC%84%9C-%EC%9B%B9-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-selenium

코드 참고했습니다

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Boaz/미니플젝2' # (최초 1회)
!pip install chromedriver-autoinstaller

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import requests
import os

import time
import chromedriver_autoinstaller

In [ ]:
chrome_path = "/content/drive/MyDrive/Boaz/미니플젝2/chromedriver"

In [ ]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off : cloab은 새창을 지원하지않기 때문에 창 없는 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

In [ ]:
### 1. 드라이브 마운트 및 패키지 설치는 생략 (기존 코드 참고)

### 2. selenium_scroll_option 함수 정의 및 이미지 수집
def selenium_scroll_option_and_collect_images(driver, save_folder, pause_time=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_attempts = 0  # 스크롤 시도 횟수
    max_attempts = 10  # 최대 시도 횟수 설정
    img_urls_collected = set()  # 수집한 이미지 URL을 중복 없이 저장

    while scroll_attempts < max_attempts:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 대기 시간 설정
        time.sleep(pause_time)

        # 이미지 요소 찾기
        images = driver.find_elements(By.CSS_SELECTOR, "img")
        for i, img in enumerate(images):
            try:
                img_url = img.get_attribute('src')
                if img_url and 'http' in img_url and img_url not in img_urls_collected:
                    img_urls_collected.add(img_url)
                    # 이미지 저장
                    img_data = requests.get(img_url).content
                    img_index = len(img_urls_collected)  # 고유한 인덱스를 사용
                    with open(f"{save_folder}/F_white_{img_index}.jpg", 'wb') as img_file:
                        img_file.write(img_data)
            except Exception as e:
                print(f"이미지 저장 중 오류 발생: {e}")

        # 스크롤 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            scroll_attempts += 1  # 동일한 높이가 여러 번 감지되면 반복 종료 조건 증가
            if scroll_attempts >= max_attempts:
                print("더 이상 새로운 콘텐츠가 로드되지 않습니다.")
                break
        else:
            scroll_attempts = 0  # 스크롤 높이가 변경되면 시도 횟수 초기화
        last_height = new_height

    return img_urls_collected

### 3. 이미지 크롤링
# 드라이버 시작 및 URL 로드
driver = webdriver.Chrome(options=chrome_options)
url = "https://www.musinsa.com/category/001?gf=F&color=1"
driver.get(url)

# 이미지 저장 폴더 설정
save_folder = "/content/drive/MyDrive/Boaz/미니플젝2/F_white"  # 원하는 경로로 설정
os.makedirs(save_folder, exist_ok=True)

# 스크롤하면서 이미지 수집 및 저장
collected_img_urls = selenium_scroll_option_and_collect_images(driver, save_folder)

# 수집된 이미지 개수 출력
print("수집된 이미지 URL 개수:", len(collected_img_urls))

# 드라이버 종료
driver.quit()
